### Logistic Regression (Classification method)

Not all labels are continuous, so sometimes we need to predict categories, which is known as classification.

Logistic Regression is one of the most basic ways to perform classification.

Examples of classifications:
- Spam VS Normal emails
- Loan Default (Y/N)
- Disease Diagnosis

(Note: Above were all examples of Binary Classification)

We can use a Confusion Matrix to evalute classification models

### Part 1

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("LogisticRegressioni").getOrCreate()

In [3]:
# Logistic Regerssion Documentation Example

In [4]:
from pyspark.ml.classification import LogisticRegression

In [7]:
egdata = spark.read.format('libsvm').load("datasets/sample_libsvm_data.txt")

In [8]:
egdata.show(2)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
+-----+--------------------+
only showing top 2 rows



In [9]:
lor_reg_model = LogisticRegression()

In [10]:
fitted_lr = lor_reg_model.fit(egdata)

In [11]:
log_summary = fitted_lr.summary

In [12]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [13]:
train,test = egdata.randomSplit([0.7,0.3])

In [14]:
trainned_model = LogisticRegression()
final = trainned_model.fit(train)

In [15]:
predictions_n_labels = final.evaluate(test)

In [16]:
predictions_n_labels.predictions.show(3)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[121,122,123...|[24.3762709822539...|[0.99999999997408...|       0.0|
|  0.0|(692,[122,123,124...|[19.4421793105573...|[0.99999999639945...|       0.0|
|  0.0|(692,[124,125,126...|[29.9410594829723...|[0.99999999999990...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 3 rows



In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [19]:
my_eval = BinaryClassificationEvaluator()

In [20]:
roc_results = my_eval.evaluate(predictions_n_labels.predictions)

In [21]:
roc_results

1.0

### Titanic Dataset Exercise

In [24]:
titanic = spark.read.csv("datasets/titanic.csv",inferSchema=True,header=True).createOrReplaceTempView("titanic")

In [25]:
df = spark.sql("SELECT * FROM titanic")

In [26]:
df.head()

Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S')

In [27]:
df.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [28]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [29]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [30]:
my_cols = df.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked'])

In [31]:
simple_drop_data = my_cols.na.drop()

In [32]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

In [33]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
# A B C -> StringIndexer -> 0 1 2
# OneHot Encode:
# KEY A B C
# Eg A: [1,0,0]
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [34]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [35]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],
                            outputCol='features')

### Pipeline

In [36]:
from pyspark.ml import Pipeline

In [37]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [38]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                            gender_encoder,embark_encoder,
                            assembler,log_reg_titanic])

In [39]:
train, test = simple_drop_data.randomSplit([0.7,0.3])

In [40]:
fit_model = pipeline.fit(train)

In [41]:
results = fit_model.transform(test)

In [42]:
# from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [43]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [46]:
results.select('Survived','prediction').show(5)

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 5 rows



In [49]:
results.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- SexIndex: double (nullable = false)
 |-- EmbarkIndex: double (nullable = false)
 |-- SexVec: vector (nullable = true)
 |-- EmbarkVec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [48]:
AUC = my_eval.evaluate(results)
AUC

0.7426470588235294

### Part 2
#### Data: customer_churn.csv

In [41]:
from pyspark.sql import SparkSession

In [42]:
spark = SparkSession.builder.appName("LR_P2").getOrCreate()

In [43]:
data = spark.read.csv("datasets/customer_churn.csv",header=True,inferSchema=True,sep=',')

In [44]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Total_Purchase: string (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: string (nullable = true)



In [49]:
data.filter(data.Churn.isNotNull()).show()

+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+



In [5]:
data.select('location').show(1)
data.show(2)

+--------------------+
|            location|
+--------------------+
|10265 Elizabeth M...|
+--------------------+
only showing top 1 row

+----------------+----------+--------------+---------------+-----+---------+-------------------+--------------------+-------+-----+
|           Names|       Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|Company|Churn|
+----------------+----------+--------------+---------------+-----+---------+-------------------+--------------------+-------+-----+
|Cameron Williams|      42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|   null| null|
|     Barkerburgh| AK 89518"|    Harvey LLC|              1| null|     null|               null|                null|   null| null|
+----------------+----------+--------------+---------------+-----+---------+-------------------+--------------------+-------+-----+
only showing top 2 rows



In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
data.na.drop(how='any').show()

+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+



In [10]:
from pyspark.sql.types import IntegerType,DoubleType
# Column type convert
data = data.withColumn("Age",data["Age"].cast(DoubleType()))

In [32]:
# This will only select one column
# data = data.selectExpr("cast(Total_Purchase as double) Total_Purchase")

In [ ]:
# data.createOrReplaceTempView("CastExample")
# data = spark.sql("SELECT STRING(Age),BOOLEAN(isGraduated),DATE(jobStartDate) from CastExample")
# data.printSchema()
# data.show(truncate=False)

In [11]:
data = data.withColumn("Total_Purchase",data["Total_Purchase"].cast(DoubleType()))

In [12]:
data = data.withColumn("Churn",data["Churn"].cast(IntegerType()))

In [13]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [14]:
from pyspark.ml.feature import VectorAssembler

In [15]:
assembler = VectorAssembler(inputCols=['Age',
                             'Total_Purchase',
                             'Years',
                             'Num_Sites'],
                             outputCol='features')

In [37]:
new_data = data.drop('Company', 'Churn').dropna()

In [38]:
new_data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|    1331 Keith Court|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|  13120 Daniel Mount|
|     Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|2016-01-19 15:31:15|      765 Tricia Row|
|   Jessica Williams|48.0|      10356.02|              0| 5.12|      8.0|2009-03

In [39]:
output = assembler.transform(new_data)

In [40]:
output.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|            features|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|[42.0,11066.8,7.2...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|[41.0,11916.22,6....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|    1331 Keith Court|[38.0,12884.75,6....|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|  13120 Daniel Mount|[42.0,8010.76,6.7...|
|     Cynthia Norton|37.0|       9191.58|       

In [17]:
final_data= output.select('features','Churn')

In [18]:
train,test = final_data.randomSplit([0.7,0.3])

In [27]:
final_data.show(5)

Py4JJavaError: An error occurred while calling o98.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<Age:double,Total_Purchase:double,Years:double,Num_Sites:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$11$$anon$1.hasNext(WholeStageCodegenExec.scala:619)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:255)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at org.apach

In [19]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Churn: integer (nullable = true)



In [25]:
# data = 
data.filter(data.Num_Sites.isNotNull())

DataFrame[Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Onboard_date: timestamp, Location: string, Company: string, Churn: int]

In [26]:
train.show(5)

Py4JJavaError: An error occurred while calling o102.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 9, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<Age:double,Total_Purchase:double,Years:double,Num_Sites:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$11$$anon$1.hasNext(WholeStageCodegenExec.scala:619)
	at org.apache.spark.sql.execution.SparkPlan$$anon

In [49]:
from pyspark.ml.classification import LogisticRegression

In [58]:
lr_churn = LogisticRegression(labelCol='churn')

In [60]:
# Fit the model to our data
fitted_churn_model = lr_churn.fit(train)

Py4JJavaError: An error occurred while calling o358.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 11, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<Age:double,Total_Purchase:double,Account_Manager_double_VectorAssembler_f3e2dd40e12f:double,Years:double,Num_Sites:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$11$$anon$1.hasNext(WholeStageCodegenExec.scala:

In [ ]:
traning_sum = fitted_churn_model.summary

In [ ]:
traning_sum.predictions.describe().show()

### Logistic Regression Part 3
#### data source: customer_churn.csv
#### comparing against: new_customers.csv

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("LR_P3").getOrCreate()

In [18]:
data = spark.read.csv("datasets/customer_churn.csv",inferSchema=True,header=True)

#### Note: 
the data read in this notebook with pyspark version 2.4.0 has some unexpect issues
which is different with the courses, so some extra cleaning steps are required (such as NULL values need to be dropped as Vector processing cannot accept NULL)

In [19]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Total_Purchase: string (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: string (nullable = true)



In [20]:
data.describe().show()

+-------+-------------+-----------------+--------------------+------------------+-----------------+------------------+--------------------+-------+-----+
|summary|        Names|              Age|      Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|Company|Churn|
+-------+-------------+-----------------+--------------------+------------------+-----------------+------------------+--------------------+-------+-----+
|  count|         1800|             1800|                1800|              1693|              900|               900|                 900|      0|    0|
|   mean|         null|41.81666666666667|   8993.604399205567|0.3337271116361488| 5.27315555555555| 8.587777777777777|                null|   null| null|
| stddev|         null|6.127560416916251|  3848.3789060162444|0.4716828819140737|1.274449013194616|1.7648355920350969|                null|   null| null|
|    min|APO AA 00585"|        AK 08955"|                   0|              

In [21]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [23]:
from pyspark.ml.feature import VectorAssembler

In [24]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [25]:
output = assembler.transform(data)

IllegalArgumentException: 'Data type string of column Age is not supported.\nData type string of column Total_Purchase is not supported.'